## Pyspark code to extract data from hadoop into dataframes and export onto S3 ##

In [1]:
##Importing required libraries
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1643525333131_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType, TimestampType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
##creating a custom schema based on incoming file. columnOrder also needs to be maintained
columns = StructType([StructField('year', IntegerType(),True),
                        StructField('month', StringType(),True),
                        StructField('day', IntegerType(),True),
                        StructField('weekday', StringType(),True),
                        StructField('hour', IntegerType(),True),
                        StructField('atm_status', StringType(),True),
                        StructField('atm_id', StringType(),True),
                        StructField('atm_manufacturer', StringType(),True),
                        StructField('atm_location', StringType(),True),
                        StructField('atm_streetname', StringType(),True),
                        StructField('atm_street_number', IntegerType(),True),
                        StructField('atm_zipcode', IntegerType(),True),
                        StructField('atm_lat', DoubleType(),True),
                        StructField('atm_lon', DoubleType(),True),
                        StructField('currency', StringType(),True),
                        StructField('card_type', StringType(),True),
                        StructField('transaction_amount', IntegerType(),True),
                        StructField('service', StringType(),True),
                        StructField('message_code', StringType(),True),
                        StructField('message_text', StringType(),True),
                        StructField('weather_lat', DoubleType(),True),
                        StructField('weather_lon', DoubleType(),True),
                        StructField('weather_city_id', IntegerType(),True),
                        StructField('weather_city_name', StringType(),True),
                        StructField('temp', DoubleType(),True),
                        StructField('pressure', IntegerType(),True),
                        StructField('humidity', IntegerType(),True),
                        StructField('wind_speed', IntegerType(),True),
                        StructField('wind_deg', IntegerType(),True),
                        StructField('rain_3h', DoubleType(),True),
                        StructField('clouds_all', IntegerType(),True),
                        StructField('weather_id', IntegerType(),True),
                        StructField('weather_main', StringType(),True),
                        StructField('weather_description', StringType(),True),
                        ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
##read csv from user/livy folder with custom schema
df=spark.read.csv("/user/livy/SRC_ATM_TRANS/part-m-00000", header = True, schema=columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
##show a few rows to validate the csv import
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency| card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-

In [13]:
### Validating Counts
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468571

In [14]:
DIM_CARD_TYPE_raw=df.select("card_type").distinct()
DIM_CARD_TYPE_raw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [59]:
DIM_DATE_raw=df.select("year","month","day","hour","weekday").distinct()
DIM_DATE_raw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

In [15]:
DIM_ATM_raw=df.select("atm_id","atm_manufacturer","atm_location","atm_streetname","atm_lat","atm_lon").distinct()
DIM_ATM_raw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [19]:
DIM_LOCATION_raw=df.select("atm_location","atm_streetname","atm_street_number","atm_zipcode","atm_lat","atm_lon").distinct()
DIM_LOCATION_raw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [20]:
FACT_ATM_TRANS_raw=df.select("year","month","day","hour","card_type","atm_id","atm_manufacturer","atm_location","atm_streetname","atm_street_number","atm_zipcode","atm_lat","atm_lon","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id","weather_main","weather_description")
FACT_ATM_TRANS_raw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468571

In [21]:
### RENAME atm_id to atm_number. atm_id will be generated later.
DIM_ATM_renamed = DIM_ATM_raw.withColumnRenamed("atm_id","atm_number")
FACT_ATM_TRANS_raw=FACT_ATM_TRANS_raw.withColumnRenamed("atm_id","atm_number")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
### generate *_id's 
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from pyspark.sql.window import Window
DIM_DATE = DIM_DATE_raw.withColumn('date_id', row_number().over(Window.orderBy(monotonically_increasing_id())))
DIM_CARD_TYPE = DIM_CARD_TYPE_raw.withColumn('card_id', row_number().over(Window.orderBy(monotonically_increasing_id())))
DIM_ATM = DIM_ATM_renamed.withColumn('atm_id', row_number().over(Window.orderBy(monotonically_increasing_id())))
DIM_LOCATION = DIM_LOCATION_raw.withColumn('location_id', row_number().over(Window.orderBy(monotonically_increasing_id())))
FACT_ATM_TRANS = FACT_ATM_TRANS_raw.withColumn('trans_id', row_number().over(Window.orderBy(monotonically_increasing_id())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
DIM_DATE.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+---+----+-------+
|year|   month|day|hour|date_id|
+----+--------+---+----+-------+
|2017| January| 23|   2|      1|
|2017|February| 26|   8|      2|
|2017|   March| 12|  13|      3|
|2017|   March| 14|  21|      4|
|2017|   March| 21|   7|      5|
|2017| January| 28|  23|      6|
|2017|February| 14|  10|      7|
|2017| January|  1|   6|      8|
|2017| January| 25|  13|      9|
|2017|February| 17|  22|     10|
|2017|February| 21|  23|     11|
|2017|   March| 28|  23|     12|
|2017|   March| 31|  18|     13|
|2017|   March| 23|  16|     14|
|2017|   March| 21|  21|     15|
|2017|   March| 27|  22|     16|
|2017|   March| 28|  10|     17|
|2017|   March| 31|   0|     18|
|2017|   March| 13|  23|     19|
|2017|February| 26|  20|     20|
|2017|   March|  4|  10|     21|
|2017|   March|  7|   3|     22|
|2017|February|  4|  23|     23|
|2017|February|  9|   3|     24|
|2017|February|  9|  10|     25|
|2017|   March|  3|  16|     26|
|2017|   March|  4|  11|     27|
|2017|   M

In [24]:
DIM_CARD_TYPE.show(13)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+
|           card_type|card_id|
+--------------------+-------+
|Visa Dankort - on-us|      1|
|  Mastercard - on-us|      2|
|         HÃƒÂ¦vekort|      3|
|            VisaPlus|      4|
|     Dankort - on-us|      5|
|        Visa Dankort|      6|
| HÃƒÂ¦vekort - on-us|      7|
|              CIRRUS|      8|
|                VISA|      9|
|             Maestro|     10|
|          MasterCard|     11|
|             Dankort|     12|
+--------------------+-------+

In [25]:
DIM_ATM.show(113)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+--------------------+--------------------+-------+-------+------+
|atm_number|atm_manufacturer|        atm_location|      atm_streetname|atm_lat|atm_lon|atm_id|
+----------+----------------+--------------------+--------------------+-------+-------+------+
|        17|             NCR|             Randers|        ÃƒËœstervold| 56.462| 10.038|     1|
|        23| Diebold Nixdorf|             Vodskov|          Vodskovvej| 57.104| 10.027|     2|
|        73|             NCR|         HÃƒÂ¸jbjerg|      Rosenvangsalle| 56.119| 10.192|     3|
|        86|             NCR|HillerÃƒÂ¸d IdrÃƒ...|          Milnersvej| 55.921| 12.299|     4|
|        18| Diebold Nixdorf|              Viborg|           Toldboden| 56.448|  9.401|     5|
|        67|             NCR|          HÃƒÂ¸jslev|      ÃƒËœsterrisvej| 56.551|   9.11|     6|
|        84|             NCR|           Svendborg|  Sankt Nicolai Gade| 55.058| 10.609|     7|
|        55|             NCR|               Arden|

In [26]:
DIM_LOCATION.show(109)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------------+-----------+-------+-------+-----------+
|        atm_location|      atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|location_id|
+--------------------+--------------------+-----------------+-----------+-------+-------+-----------+
|             Kolding|            Vejlevej|              135|       6000| 55.505|  9.457|          1|
|      Skelagervej 15|         Skelagervej|               15|       9000| 57.023|  9.891|          2|
|    Intern HolbÃƒÂ¦k|         Slotsvolden|                7|       4300| 55.718| 11.704|          3|
|              Odense|        FÃƒÂ¦lledvej|                3|       5000| 55.394|  10.37|          4|
|               Ikast| RÃƒÂ¥dhusstrÃƒÂ¦det|               12|       7430| 56.139|  9.154|          5|
|             Randers|        ÃƒËœstervold|               16|       8900| 56.462| 10.038|          6|
|             Esbjerg|        Strandbygade|               20|       6700| 55.468| 

In [27]:
FACT_ATM_TRANS.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+------------------+----------+----------------+--------------+-------------------+-----------------+-----------+-------+-------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+--------------------+--------+
|year|  month|day|hour|         card_type|atm_number|atm_manufacturer|  atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main| weather_description|trans_id|
+----+-------+---+----+------------------+----------+----------------+--------------+-------------------+-----------------+-----------+-------+-------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+--------------------+--------+
|2017|January|  1|   0|        MasterCard|         2|             NCR|      Vejgaard|         Had

In [61]:
### Create TimeStamp in Date Dimention Table from year,month,date and hour
from pyspark.sql.functions import *
from pyspark.sql import functions as func
### Convert month into numeric format
DIM_DATE=DIM_DATE.withColumn("month",from_unixtime(unix_timestamp(col("month"),'MMM'),'MM'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
DIM_DATE.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+----+-------+
|year|month|day|hour|date_id|
+----+-----+---+----+-------+
|2017|   01| 23|   2|      1|
|2017|   02| 26|   8|      2|
|2017|   03| 12|  13|      3|
|2017|   03| 14|  21|      4|
|2017|   03| 21|   7|      5|
+----+-----+---+----+-------+
only showing top 5 rows

In [62]:
##create date field from year,month,day
cols=["year","month","day"]
DIM_DATE=DIM_DATE.withColumn("date",concat_ws("-",*cols).cast("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
DIM_DATE.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+----+-------+----------+
|year|month|day|hour|date_id|      date|
+----+-----+---+----+-------+----------+
|2017|   01| 23|   2|      1|2017-01-23|
|2017|   02| 26|   8|      2|2017-02-26|
|2017|   03| 12|  13|      3|2017-03-12|
|2017|   03| 14|  21|      4|2017-03-14|
|2017|   03| 21|   7|      5|2017-03-21|
+----+-----+---+----+-------+----------+
only showing top 5 rows

In [63]:
### Create full_date_time from date and hour 
DIM_DATE=DIM_DATE.withColumn("full_date_time",func.date_format(func.to_timestamp(func.concat("date","hour"),"yyyy-MM-ddHH"),"yyyy-MM-dd hh:ss:SSa"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
DIM_DATE.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+----+-------+----------+--------------------+
|year|month|day|hour|date_id|      date|      full_date_time|
+----+-----+---+----+-------+----------+--------------------+
|2017|   01| 23|   2|      1|2017-01-23|2017-01-23 02:00:...|
|2017|   02| 26|   8|      2|2017-02-26|2017-02-26 08:00:...|
|2017|   03| 12|  13|      3|2017-03-12|2017-03-12 01:00:...|
+----+-----+---+----+-------+----------+--------------------+
only showing top 3 rows

In [64]:
###revert the month from numeric back to string
DIM_DATE=DIM_DATE.withColumn("month",from_unixtime(unix_timestamp(col("month"),'MM'),'MMM'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
DIM_DATE.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+----+-------+----------+--------------------+
|year|month|day|hour|date_id|      date|      full_date_time|
+----+-----+---+----+-------+----------+--------------------+
|2017|  Jan| 23|   2|      1|2017-01-23|2017-01-23 02:00:...|
|2017|  Feb| 26|   8|      2|2017-02-26|2017-02-26 08:00:...|
|2017|  Mar| 12|  13|      3|2017-03-12|2017-03-12 01:00:...|
+----+-----+---+----+-------+----------+--------------------+
only showing top 3 rows

In [65]:
##cast as timestamptype
DIM_DATE=DIM_DATE.withColumn('full_date_time',func.unix_timestamp('full_date_time','yyyy-MM-dd HH:mm:ss').cast(TimestampType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
DIM_DATE.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+----+--------+-------+----------+-------------------+
|year|month|day|hour| weekday|date_id|      date|     full_date_time|
+----+-----+---+----+--------+-------+----------+-------------------+
|2017|  Jan|  5|  21|Thursday|      1|2017-01-05|2017-01-05 09:00:00|
|2017|  Jan| 22|  15|  Sunday|      2|2017-01-22|2017-01-22 03:00:00|
|2017|  Apr|  7|   9|  Friday|      3|2017-04-07|2017-04-07 09:00:00|
+----+-----+---+----+--------+-------+----------+-------------------+
only showing top 3 rows

In [57]:
### Perform the same transformation on FACT_ATM_TRANS as in DIM_DATE for the upcoming join function
### Needed because in DIM_DATE the transformation is Jaunary ---> 01 ---> Jan
FACT_ATM_TRANS=FACT_ATM_TRANS.withColumn("month",from_unixtime(unix_timestamp(col("month"),'MMM'),'MM'))
FACT_ATM_TRANS=FACT_ATM_TRANS.withColumn("month",from_unixtime(unix_timestamp(col("month"),'MM'),'MMM'))
FACT_ATM_TRANS.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+----+----------+----------+----------------+------------+-------------------+-----------------+-----------+-------+-------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+--------------------+--------+
|year|month|day|hour| card_type|atm_number|atm_manufacturer|atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main| weather_description|trans_id|
+----+-----+---+----+----------+----------+----------------+------------+-------------------+-----------------+-----------+-------+-------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+--------------------+--------+
|2017|  Jan|  1|   0|MasterCard|         2|             NCR|    Vejgaard|         Hadsundvej|               20|       9000| 57.043|  

In [67]:
### REARRANGING COLUMNS [ Dimensions Table]
DIM_DATE = DIM_DATE.select("date_id","full_date_time","year","month","day","hour","weekday")
DIM_CARD_TYPE = DIM_CARD_TYPE.select("card_id","card_type")
DIM_ATM = DIM_ATM.select("atm_id","atm_number","atm_manufacturer","atm_location","atm_streetname","atm_lat","atm_lon")
DIM_LOCATION = DIM_LOCATION.select("location_id","atm_location","atm_streetname","atm_street_number","atm_zipcode","atm_lat","atm_lon")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
### Perform join to import *_id's to FACT_ATM_TRANS
FACT_ATM_TRANS=FACT_ATM_TRANS.join(DIM_DATE,on=['year','month','day','hour'],how="left")
FACT_ATM_TRANS.show(3)
FACT_ATM_TRANS=FACT_ATM_TRANS.join(DIM_CARD_TYPE,on=['card_type'],how="left")
FACT_ATM_TRANS=FACT_ATM_TRANS.join(DIM_ATM,on=['atm_number','atm_manufacturer'],how="left")
FACT_ATM_TRANS=FACT_ATM_TRANS.join(DIM_LOCATION,on=['atm_location','atm_streetname','atm_lat','atm_lon'],how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+----------+----------+----------------+------------+-------------------+-----------------+-----------+-------+-------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+--------------------+--------+-------+--------------+-------+
|year|  month|day|hour| card_type|atm_number|atm_manufacturer|atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main| weather_description|trans_id|date_id|full_date_time|weekday|
+----+-------+---+----+----------+----------+----------------+------------+-------------------+-----------------+-----------+-------+-------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+--------------------+--------+-------+--------------+-------+
|2017|January|  1|   0|MasterCard|

In [69]:
FACT_ATM_TRANS.drop("year","month","day","hour","card_type","atm_number","atm_manufacturer","atm_location","atm_streetname","atm_street_number","atm_zipcode","atm_lat","atm_lon")
###Rearranging Fact table and rename columns per target schema
FACT_ATM_TRANS = FACT_ATM_TRANS.select("trans_id","atm_id","location_id","date_id","card_id","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id","weather_main","weather_description")
FACT_ATM_TRANS = FACT_ATM_TRANS.withColumnRenamed("location_id","weather_loc_id")
FACT_ATM_TRANS = FACT_ATM_TRANS.withColumnRenamed("card_id","card_type_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
DIM_CARD_TYPE = DIM_CARD_TYPE.withColumnRenamed("card_id","card_type_id")
DIM_LOCATION = DIM_LOCATION.withColumnRenamed("atm_street_number","street_number")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
### Perform join to import location_id to DIM_ATM
DIM_ATM=DIM_ATM.join(DIM_LOCATION,on=['atm_location','atm_streetname','atm_lat','atm_lon'],how="left")
DIM_ATM=DIM_ATM.select("atm_id","atm_number","atm_manufacturer","location_id")
DIM_ATM=DIM_ATM.withColumnRenamed("location_id","atm_location_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
### Rename according to target schema
DIM_LOCATION=DIM_LOCATION.withColumnRenamed("atm_location","location")
DIM_LOCATION=DIM_LOCATION.withColumnRenamed("atm_streetname","streetname")
DIM_LOCATION=DIM_LOCATION.withColumnRenamed("atm_zipcode","zipcode")
DIM_LOCATION=DIM_LOCATION.withColumnRenamed("atm_lat","lat")
DIM_LOCATION=DIM_LOCATION.withColumnRenamed("atm_lon","lon")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
### Verify the columns - per target schema
DIM_DATE.printSchema()
DIM_ATM.printSchema()
DIM_LOCATION.printSchema()
DIM_CARD_TYPE.printSchema()
FACT_ATM_TRANS.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date_id: integer (nullable = true)
 |-- full_date_time: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- weekday: string (nullable = true)

root
 |-- atm_id: integer (nullable = true)
 |-- atm_number: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location_id: integer (nullable = true)

root
 |-- location_id: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- streetname: string (nullable = true)
 |-- street_number: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)

root
 |-- card_type_id: integer (nullable = true)
 |-- card_type: string (nullable = true)

root
 |-- trans_id: integer (nullable = true)
 |-- atm_id: integer (nullable = true)
 |-- weather_loc_id: integer (nullable = true)
 |-- date_id: integer (n

In [75]:
### validating final counts 
DIM_DATE.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

In [76]:
DIM_ATM.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [77]:
DIM_LOCATION.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [78]:
DIM_CARD_TYPE.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [79]:
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468571

In [81]:
##writing to s3. Bucket should be created in advance of running this function
DIM_CARD_TYPE.write.csv('s3a://upgrad-etlproject-vinay/dim_card_type/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o1114.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:165)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at or

In [82]:
DIM_LOCATION.write.csv('s3a://upgrad-etlproject-vinay/dim_location/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o1155.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:165)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at or

In [83]:
DIM_ATM.write.csv('s3a://upgrad-etlproject-vinay/dim_atm/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o1196.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:165)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at or

In [84]:
DIM_DATE.write.csv('s3a://upgrad-etlproject-vinay/dim_date/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o1237.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:165)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at or

In [85]:
FACT_ATM_TRANS.write.csv('s3a://upgrad-etlproject-vinay/fact_atm_trans/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o1278.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:165)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at or